In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py


from holodeck import plot, detstats
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC
import holodeck as holo

import hasasia.sim as hsim

# Choose a Parameter Space

In [ ]:
SHAPE = (20,15,25)
NREALS = 30
NFREQS = 40
NLOUDEST = 10

 # construct a param_space instance, note that `nsamples` and `seed` don't matter here for how we'll use this
pspace = holo.param_spaces.PS_Uniform_09A(holo.log, nsamples=1, sam_shape=SHAPE, seed=None)

# get the parameter names from this library-space
param_names = pspace.param_names
num_pars = len(pspace.param_names)
print(f"{num_pars=} :: {param_names=}")

# choose each parameter to be half-way across the range provided by the library
pars = 0.5 * np.ones(num_pars)
params = pspace.normalized_params(pars)
# params = pars * pspace.param_samples
print(f"{pars=}")
print(f"{params=}")

# construct `sam` and `hard` instances based on these parameters,
# using otherwise all default parameters for this library
sam, hard = pspace.model_for_normalized_params(pars)
if isinstance(hard, holo.hardening.Fixed_Time_2PL_SAM):
    hard_name = 'Fixed Time'
elif isinstance(hard, holo.hardening.Hard_GW):
    hard_name = 'GW Only'
# sam, hard = pspace.model_for_params(pspace.normalized_params(pars)) #this is way slower, but why??

# run this model, retrieving binary parameters and the GWB
data = holo.librarian.run_model(sam, hard, NREALS, NFREQS, NLOUDEST, 
                                gwb_flag=False, singles_flag=True, params_flag=True, details_flag=True)
print(f"retrieved data: {data.keys()=}")

Question: What are bin_params and gwb_params?

In [ ]:
fobs_cents = data['fobs_cents']
hc_ss_mid = data['hc_ss']
hc_bg_mid = data['hc_bg']
sspar_mid = sings.all_sspars(fobs_cents, data['sspar'])
bgpar_mid = data['bgpar']
binpar_names = sings.par_names
# params = pars*pspace.params
print(f"{hc_ss_mid.shape=}")
print(f"{bgpar_mid.shape=}")
print(f"{sspar_mid.shape=}")
print(f"{param_names=}")
print(f"{params=}")
print(f"{binpar_names=}")

# Plot hc vs. bin pars

In [ ]:
print(sings.par_labels)
print(sings.par_units)

In [ ]:
def draw_sample_text(fig, params, param_names, 
                     xx=0.1, yy=-0.025, fontsize=10, color='k'):
    text = ''
    for pp, name in enumerate(param_names):
        text = text+"'%s'=%.2e, " % (name, params[name])
        # if pp == int(len(param_names)/2):
        #     text = text+'\n'
    fig.text(xx, yy, text, fontsize=fontsize, color=color, alpha=0.75)


def draw_hs_vs_par(ax, xx_ss=None, yy_ss=None, xx_bg=None, yy_bg=None, color_ss='r', color_bg='k', fast_ss=True, 
                   show_medians = False, show_ci=False, show_reals=True):
    if show_reals:
        if (xx_ss is not None) and (yy_ss is not None):
            if fast_ss:
                ax.scatter(xx_ss.flatten(), yy_ss.flatten(), marker='o', s=15, alpha=0.1, color=color_ss)
            else:
                colors = cm.rainbow(np.linspace(0,1,len(yy_ss[0])))
                for rr in range(len(yy_ss[0])):
                    ax.scatter(xx_ss[:,rr,:].flatten(), yy_ss[:,rr,:].flatten(), marker='o', s=10, alpha=0.1, color=colors[rr])
        if (xx_bg is not None) and (yy_bg is not None):
            ax.scatter(xx_bg.flatten(), yy_bg.flatten(), marker='x', s=15, alpha=0.1, color=color_bg)
    # if show_medians:
    #     if (xx_ss is not None) and (yy_ss is not None):
    #         ax.plot(np.median(xx_ss, axis=())
    #     if (xx_bg is not None) and (yy_bg is not None):




def plot_hs_vs_binpars(fobs_cents, hc_ss, hc_bg, sspar, bgpar, color_ss='r', color_bg='k', fast_ss=True):
    """ plot mtot (0), mrat (1), redz_init (2), dc_final (4), sepa_final(5), angs_final(6)"""
    colors = cm.rainbow(np.linspace(0,1,len(hc_ss[0])))
    idx = [0,1,2,4,5,6]

    labels = sings.par_labels[idx]
    units = sings.par_units[idx]
    xx_ss = sspar[idx]*units[:,np.newaxis,np.newaxis,np.newaxis]
    xx_bg = bgpar[idx]*units[:,np.newaxis,np.newaxis]
    print(f"{xx_ss.shape=}")
    print(f"{xx_bg.shape=}")

    yy_ss = hc_ss
    yy_bg = hc_bg
    print(f"{yy_ss.shape=}")
    print(f"{yy_bg.shape=}")


    fig, axs = holo.plot.figax(
        nrows=2, ncols=3, sharey=True, figsize=(12,6))
    for ax in axs[:,0]:
        ax.set_ylabel(holo.plot.LABEL_CHARACTERISTIC_STRAIN)
    for ii, ax in enumerate(axs.flatten()):
        ax.set_xlabel(labels[ii])
        draw_hs_vs_par(ax, xx_ss[ii], yy_ss, xx_bg[ii], yy_bg, color_ss, color_bg, fast_ss, colors)
        
    draw_sample_text(fig, params, param_names, xx=0.1, yy=-0.05, fontsize=12)
    fig.tight_layout()
    return fig

fig = plot_hs_vs_binpars(fobs_cents, hc_ss_mid, hc_bg_mid, sspar_mid, bgpar_mid, fast_ss=True)

# Calculate detstats

In [ ]:
NPSRS = 60
SIGMA = 1e-7
NSKIES = 25
THRESH = 0.5
DUR = holo.librarian.DEF_PTA_DUR

In [ ]:

def detect_pspace_model(data, dur=DUR,
                        npsrs=NPSRS, sigma=SIGMA, nskies=NSKIES, thresh=THRESH):
    fobs_cents = data['fobs_cents']
    hc_ss = data['hc_ss']
    hc_bg = data['hc_bg']
    dsdata = detstats.detect_pspace_model(fobs_cents, hc_ss, hc_bg, dur,
                        npsrs, sigma, nskies, thresh)
    return dsdata

dsdata_mid= detect_pspace_model(data)
print(f"{dsdata_mid.keys()=}")

# Plot everything vs freqs

In [ ]:
arr = np.random.uniform(0,1,100).reshape(5,20)
print(arr.shape)
app = np.random.uniform(0,1, 20)
tot = np.append(arr, app).reshape(6,20)
print(tot.shape)
print(np.all(arr[2] == tot[2]))

In [ ]:
dp_ss = dsdata_mid['dp_ss'] 
dp_bg = dsdata_mid['dp_bg']
snr_ss = dsdata_mid['snr_ss']
snr_bg = dsdata_mid['snr_bg']
print(f"{dp_ss.shape=}, {dp_bg.shape=}, {snr_ss.shape=}, {snr_bg.shape=}")

In [ ]:
def draw_par_vs_freq(ax, xx, xx_ss, yy_ss, xx_bg, yy_bg, 
                     color_ss, color_bg, colors,
                     fast, show_reals, nreals, nloudest):
    if show_reals:
        if fast:
            ax.scatter(xx_ss.flatten(), yy_ss.flatten(), marker='o', alpha=0.1, color=color_ss)
            ax.scatter(xx_bg.flatten(), yy_bg.flatten(), marker='x', alpha=0.1, color=color_bg)
        else:
            for rr in range(nreals):
                for ll in range(nloudest):
                    edgecolor='k' if ll==0 else None
                    ax.scatter(xx, yy_ss[:,rr,ll], marker='o', s=15, alpha=0.1, color=colors[rr], edgecolor=edgecolor)
                ax.plot(xx, yy_bg[:,rr], marker='x', alpha=0.1, color=colors[rr])


def draw_snr_vs_freqs(ax, xx, snr_ss, snr_bg, 
             color_ss, color_bg, ls_ss, ls_bg, lw_ss, lw_bg, 
             colors, fast, nfreqs, nreals, nskies, nloudest,):
    
    xx_snr = np.repeat(xx, nreals*nskies*nloudest).reshape(nfreqs, nreals, nskies, nloudest)
    if fast:
        ax.scatter(xx_snr.flatten(), snr_ss.flatten(), marker='o', alpha=0.1, color=color_ss)
        for rr in range(nreals):
            ax.axhline(snr_bg[rr], ls=ls_bg, lw=lw_bg, alpha=0.1, color=color_bg)
    else:
        xx_skies = np.repeat(xx, nskies).reshape(nfreqs, nskies)
        for rr in range(nreals):
            for ll in range(nloudest):
                edgecolor = 'k' if ll==0 else None
                ax.scatter(xx_skies.flatten(), snr_ss[:,rr,:,ll].flatten(), marker='o', s=15, alpha=0.1, color=colors[rr], edgecolor=edgecolor)
            ax.axhline(snr_bg[rr], ls=ls_bg, lw=lw_bg, alpha=0.1, color=color_bg) 

def draw_dp_vs_freqs(ax, dp_ss, dp_bg,
            color_ss, color_bg, ls_ss, ls_bg, lw_ss, lw_bg,
            nreals, nskies):
    for rr in range(nreals):
        ax.axhline(dp_bg[rr], ls=ls_bg, lw=lw_bg, alpha=0.25, color=color_bg)
        for ss in range(nskies):
            ax.axhline(dp_ss[rr,ss], ls=ls_ss, lw=lw_ss, alpha=0.1, color=color_ss)

def plot_everything_vs_freqs(fobs_cents, hc_ss, hc_bg, sspar, bgpar, dp_ss, dp_bg, snr_ss, snr_bg,
                             color_ss='r', color_bg='k', ls_ss = ':', ls_bg = '--', lw_ss = 2, lw_bg = 2, 
                             fast=True, show_reals=True):
    """ plot mtot (0), mrat (1), redz_init (2), 
    dc_final (4), sepa_final(5), angs_final(6),
    hs, snr, dp"""
    colors = cm.rainbow(np.linspace(0,1,len(hc_ss[0])))
    idx = [0,1,2,4,5,6]
    shape = snr_ss.shape
    nfreqs, nreals, nskies, nloudest = shape[0], shape[1], shape[2], shape[3]
    xx = fobs_cents*YR
    xx_ss = np.repeat(xx, nreals*nloudest).reshape(nfreqs, nreals, nloudest)
    xx_bg = np.repeat(xx, nreals).reshape(nfreqs, nreals)

    labels = np.append(sings.par_labels[idx], 
                       np.array([plot.LABEL_CHARACTERISTIC_STRAIN, 'SNR', 'Detection Probability']))
    units = sings.par_units[idx]
    yy_ss = sspar[idx]*units[:,np.newaxis,np.newaxis,np.newaxis] # shape 6, F,R,L
    yy_ss = np.append(yy_ss, hc_ss).reshape(7, nfreqs, nreals, nloudest) # shape 7, F,R,L
    yy_bg = bgpar[idx]*units[:,np.newaxis,np.newaxis] # shape 6,F,R
    yy_bg = np.append(yy_bg, hc_bg).reshape(7, nfreqs, nreals) # shape 7,F,R
    print(f"{yy_ss.shape=}")
    print(f"{yy_bg.shape=}")

    print(f"{snr_ss.shape=}")
    print(f"{dp_ss.shape=}")


    fig, axs = holo.plot.figax(
        nrows=3, ncols=3, sharex=True, figsize=(15,10))
    for ax in axs[-1]:
        ax.set_xlabel(holo.plot.LABEL_GW_FREQUENCY_YR)

    # plot all pars and hs
    for ii,ax in enumerate(axs.flatten()[:7]):
        print('plotting', labels[ii])
        ax.set_ylabel(labels[ii])
        draw_par_vs_freq(ax, xx, xx_ss, yy_ss[ii], xx_bg, yy_bg[ii], 
                         color_ss, color_bg, colors,
                         fast, show_reals, nreals, nloudest)
        
    # plot snr, snr_ss = (F,R,L), snr_bg = (R)
    ii=7
    ax = axs.flatten()[ii]
    ax.set_ylabel(labels[ii])
    print('plotting', labels[ii])

    draw_snr_vs_freqs(ax, xx, snr_ss, snr_bg, 
             color_ss, color_bg, ls_ss, ls_bg, lw_ss, lw_bg,
             colors, fast, nfreqs, nreals, nskies, nloudest,)
    
    # plot detection probability, dp_ss = (R,S), dp_bg = (R)
    ii=8
    ax = axs.flatten()[ii]
    ax.set_ylabel(labels[ii])
    print('plotting', labels[ii])

    draw_dp_vs_freqs(ax, dp_ss, dp_bg,
            color_ss, color_bg, ls_ss, ls_bg, lw_ss, lw_bg,
            nreals, nskies)
    
    draw_sample_text(fig, params, param_names, xx=0.1, yy=-0.05, fontsize=12)    
    fig.tight_layout()
    return fig





fig=plot_everything_vs_freqs(fobs_cents, hc_ss_mid, hc_bg_mid, sspar_mid, bgpar_mid, dp_ss, dp_bg, snr_ss, snr_bg,
                             fast=False)

# Compare Simulations

In [ ]:
print(pspace.param_samples)

In [ ]:
# Choose a parameter that we're going to vary
target_param = 'hard_time'      # the name of the parameter, has to exist in `param_names`
params_list = [0.0, 0.5, 1.0]   # the values we'll check
param_idx = param_names.index(target_param)

data_hard_time = []
params_hard_time = []
for ii, par in enumerate(params_list):
    pars[param_idx] = par
    print(f"{ii=}, {pars=}")
    _params = pspace.normalized_params(pars)
    # _params = pspace.param_samples[0]*pars
    params_hard_time.append(_params)
    # construct `sam` and `hard` instances based on these parameters
    sam, hard = pspace.model_for_normalized_params(pars)
    # run this model, retrieving binary parameters and the GWB
    _data = holo.librarian.run_model(sam, hard, NREALS, NFREQS, 
                                     gwb_flag=False, singles_flag=True, params_flag=True, details_flag=True)
    data_hard_time.append(_data)

In [ ]:
def plot_everything_vs_freqs_from_data(data, fast=True, color_ss='r', color_bg='k'):
    dsdata = detect_pspace_model(data)
    sspar = sings.all_sspars(data['fobs_cents'], data['sspar'])
    fig=plot_everything_vs_freqs(
        data['fobs_cents'], data['hc_ss'], data['hc_bg'], sspar, data['bgpar'],
        dsdata['dp_ss'], dsdata['dp_bg'], dsdata['snr_ss'], dsdata['snr_bg'],
        color_ss=color_ss, color_bg=color_bg, fast=fast, )
    return fig

In [ ]:
for dat in data_hard_time:
    plot_everything_vs_freqs_from_data(dat)

In [ ]:
def draw_everything_model(fig, axs, fobs_cents, hc_ss, hc_bg, sspar, bgpar, dp_ss, dp_bg, snr_ss, snr_bg,
                          color_ss='r', color_bg='k', ls_ss = ':', ls_bg = '--', lw_ss = 2, lw_bg = 2, 
                          fast=True, show_reals=True): 
    colors = cm.rainbow(np.linspace(0,1,len(hc_ss[0])))
    idx = [0,1,2,4,5,6]
    shape = snr_ss.shape
    nfreqs, nreals, nskies, nloudest = shape[0], shape[1], shape[2], shape[3]
    xx = fobs_cents*YR
    xx_ss = np.repeat(xx, nreals*nloudest).reshape(nfreqs, nreals, nloudest)
    xx_bg = np.repeat(xx, nreals).reshape(nfreqs, nreals)

    units = sings.par_units[idx]
    yy_ss = sspar[idx]*units[:,np.newaxis,np.newaxis,np.newaxis] # shape 6, F,R,L
    yy_ss = np.append(yy_ss, hc_ss).reshape(7, nfreqs, nreals, nloudest) # shape 7, F,R,L
    yy_bg = bgpar[idx]*units[:,np.newaxis,np.newaxis] # shape 6,F,R
    yy_bg = np.append(yy_bg, hc_bg).reshape(7, nfreqs, nreals) # shape 7,F,R

    # plot all pars and hs
    for ii,ax in enumerate(axs.flatten()[:7]):
        draw_par_vs_freq(ax, xx, xx_ss, yy_ss[ii], xx_bg, yy_bg[ii], 
                        color_ss, color_bg, colors,
                        fast, show_reals, nreals, nloudest)
        
    # plot snr, snr_ss = (F,R,L), snr_bg = (R)
    ii=7
    ax = axs.flatten()[ii]
    draw_snr_vs_freqs(ax, xx, snr_ss, snr_bg, 
            color_ss, color_bg, ls_ss, ls_bg, lw_ss, lw_bg,
            colors, fast, nfreqs, nreals, nskies, nloudest,)
    
    # plot detection probability, dp_ss = (R,S), dp_bg = (R)
    ii=8
    ax = axs.flatten()[ii]
    draw_dp_vs_freqs(ax, dp_ss, dp_bg,
            color_ss, color_bg, ls_ss, ls_bg, lw_ss, lw_bg,
            nreals, nskies)
    
    return fig


def draw_three_models(
        data, params, hard_name, shape, target_param,
        datcolor_ss = np.array(['limegreen', 'cornflowerblue', 'tomato']),
        datcolor_bg = np.array(['#003300', 'darkblue', 'darkred']),
        datlw = np.array([3,4,5]),
        dattext_yy = np.array([-0.02, -0.05, -0.08])):
    
    fobs_cents = data[0]['fobs_cents']
    fig, axs = holo.plot.figax(
        nrows=3, ncols=3, sharex=True, figsize=(15,10))

    idx = [0,1,2,4,5,6]
    labels = np.append(sings.par_labels[idx], 
                        np.array([plot.LABEL_CHARACTERISTIC_STRAIN, 
                                  'SNR', 'Detection Probability']))
        
    for ax in axs[-1]:
        ax.set_xlabel(holo.plot.LABEL_GW_FREQUENCY_YR)
    for ii,ax in enumerate(axs.flatten()):
        ax.set_ylabel(labels[ii])


    for ii, dat in enumerate(data):
        print(f'on dat {ii}')
        dsdat = detect_pspace_model(dat)
        sspar = sings.all_sspars(fobs_cents, dat['sspar'])
        fig = draw_everything_model(fig, axs, fobs_cents, dat['hc_ss'], dat['hc_bg'], 
                                    sspar, dat['bgpar'], dsdat['dp_ss'], dsdat['dp_bg'],
                                    dsdat['snr_ss'], dsdat['snr_bg'], 
                                    color_ss=datcolor_ss[ii], color_bg=datcolor_bg[ii],
                                    lw_bg = datlw[ii], lw_ss = datlw[ii]) 
        draw_sample_text(fig, params[ii], param_names, color=datcolor_bg[ii], 
                         yy=dattext_yy[ii], xx=0, fontsize=12)
    fig.suptitle("%s, %s, Varying '%s'" % (hard_name, str(shape), target_param))
    fig.tight_layout()

    return fig

fig = draw_three_models(data = data_hard_time, params = params_hard_time,
                        hard_name=hard_name, shape=sam.shape, target_param=target_param)

In [ ]:
print(params_hard_time[0])

In [ ]:
print(data_hard_time[0].keys())
print(data['number'].shape)

In [ ]:
def draw_hs_vs_binpars(fig, axs, hc_ss, hc_bg, sspar, bgpar, color_ss='r', color_bg='k', fast_ss=True):
    """ plot mtot (0), mrat (1), redz_init (2), dc_final (4), sepa_final(5), angs_final(6)"""
    colors = cm.rainbow(np.linspace(0,1,len(hc_ss[0])))
    idx = [0,1,2,4,5,6]

    labels = sings.par_labels[idx]
    units = sings.par_units[idx]
    xx_ss = sspar[idx]*units[:,np.newaxis,np.newaxis,np.newaxis]
    xx_bg = bgpar[idx]*units[:,np.newaxis,np.newaxis]

    yy_ss = hc_ss
    yy_bg = hc_bg

    for ax in axs[:,0]:
        ax.set_ylabel(holo.plot.LABEL_CHARACTERISTIC_STRAIN)
    for ii, ax in enumerate(axs.flatten()):
        ax.set_xlabel(labels[ii])
        draw_hs_vs_par(ax, xx_ss[ii], yy_ss, xx_bg[ii], yy_bg, color_ss, color_bg, fast_ss, colors)
        
    return fig

def draw_three_hs_vs_binpars(data, params,
                             hard_name, shape, target_param,
                             datcolor_ss = np.array(['limegreen', 'cornflowerblue', 'tomato']),
                             datcolor_bg = np.array(['#003300', 'darkblue', 'darkred']),
                             dattext_yy = np.array([-0.02, -0.05, -0.08])):
    fobs_cents = data[0]['fobs_cents']
    
    fig, axs = holo.plot.figax(
        nrows=2, ncols=3, sharey=True, figsize=(12,6)
        )
    
    for ii, dat in enumerate(data):
        sspar = sings.all_sspars(fobs_cents, dat['sspar'])
        
        draw_hs_vs_binpars(fig, axs, dat['hc_ss'], dat['hc_bg'], sspar, dat['bgpar'], fast_ss=True,
                        color_bg = datcolor_bg[ii], color_ss=datcolor_ss[ii])
        draw_sample_text(fig, params[ii], param_names, color=datcolor_bg[ii], 
                         yy=dattext_yy[ii], xx=0, fontsize=9.5)
    fig.suptitle("%s, %s, Varying '%s'" % (hard_name, str(shape), target_param))
    fig.tight_layout()

    return fig

fig = draw_three_hs_vs_binpars(data_hard_time, params_hard_time, 
                               hard_name, sam.shape, target_param)


In [ ]:
print(param_names)

# Vary Parameters!!!!!!!

In [ ]:
def vary_parameter(
                   target_param,    # the name of the parameter, has to exist in `param_names`
                   params_list = [0.0, 0.5, 1.0],  # the values we'll check
                   pspace = holo.param_spaces.PS_Uniform_09A(holo.log, nsamples=1, sam_shape=SHAPE, seed=None),
                   pars=None 
                   ):
    # get the parameter names from this library-space
    param_names = pspace.param_names
    num_pars = len(pspace.param_names)
    print(f"{num_pars=} :: {param_names=}")

    # choose each parameter to be half-way across the range provided by the library
    if pars is None:
        pars = 0.5 * np.ones(num_pars)  
    # Choose parameter to vary
    param_idx = param_names.index(target_param)

    data = []
    params = []
    for ii, par in enumerate(params_list):
        pars[param_idx] = par
        print(f"{ii=}, {pars=}")
        # _params = pspace.param_samples[0]*pars
        _params = pspace.normalized_params(pars)
        params.append(_params)
        # construct `sam` and `hard` instances based on these parameters
        sam, hard = pspace.model_for_params(_params)
        if isinstance(hard, holo.hardening.Fixed_Time_2PL_SAM):
            hard_name = 'Fixed Time'
        elif isinstance(hard, holo.hardening.Hard_GW):
            hard_name = 'GW Only'
        # run this model, retrieving binary parameters and the GWB
        _data = holo.librarian.run_model(sam, hard, NREALS, NFREQS, 
                                        gwb_flag=False, singles_flag=True, params_flag=True, details_flag=True)
        data.append(_data)
    return (data, params, hard_name, sam.shape, target_param)

## 'hard_time'

In [ ]:
rv_hard_time = vary_parameter('hard_time') # save for later so we don't have to recalculate, if 
fig = draw_three_models(*rv_hard_time)
fig = draw_three_hs_vs_binpars(*rv_hard_time)

## 'gsmf_phi0'

In [ ]:
rv_gsmf_phi0 = vary_parameter('gsmf_phi0') 
fig = draw_three_models(*rv_gsmf_phi0)
fig = draw_three_hs_vs_binpars(*rv_gsmf_phi0)

## 'gsmf_mchar0_log10'

In [ ]:
rv_gsmf_mchar0_log10 = vary_parameter('gsmf_mchar0_log10') 
fig = draw_three_models(*rv_gsmf_mchar0_log10)
fig = draw_three_hs_vs_binpars(*rv_gsmf_mchar0_log10)

## 'mmb_mamp_log10'

In [ ]:
rv_mmb_mamp_log10 = vary_parameter('mmb_mamp_log10') 
fig = draw_three_models(*rv_mmb_mamp_log10)
fig = draw_three_hs_vs_binpars(*rv_mmb_mamp_log10)

## 'mmb_scatter_dex'

In [ ]:
rv_mmb_scatter_dex = vary_parameter('mmb_scatter_dex') 
fig = draw_three_models(*rv_mmb_scatter_dex)
fig = draw_three_hs_vs_binpars(*rv_mmb_scatter_dex)

## 'hard_gamma_inner'

In [ ]:
rv_hard_gamma_inner = vary_parameter('hard_gamma_inner') 
fig = draw_three_models(*rv_hard_gamma_inner)
fig = draw_three_hs_vs_binpars(*rv_hard_gamma_inner)